In [42]:
!pip install unidecode

^C
ERROR: Operation cancelled by user


In [43]:
!pip install rouge_score

^C
ERROR: Operation cancelled by user


In [44]:
import random
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
from rouge_score import rouge_scorer

In [45]:
import os, shutil
import re
from transformers import BertTokenizer
import nltk
import regex
import unidecode
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
files = [
         '/kaggle/input/persian-wikipedia-dataset/Persian-WikiText-6.txt',
        ]

text = ''
for file in files:
    with open(file, 'r', encoding='utf-8') as f:
        text += (f.read())

In [47]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  52303455


In [48]:
print(text[142000:145000])

تون ها، به رنگ سنگی که ستون از آن تراش خورده اکتفا شده است، در حالی که در کاخ کوروش هخامنشی، تکنیک صیقل و تراش سنگ ها این امکان را به هنرمند داده است که از سنگ هایی با دو رنگ متفاوتِ سیاه و سفید استفاده کند. مهم ترین ویژگی در تمام محوطه ها و کاخ های هخامنشی در این آثار، استفاده از سنگ است، اما کاخ کوروش هخامنشی در چرخاب برازجان تنها نمونهٔ از معماری این عصر است که در بنای آن، هم از سنگ استفاده شده است و هم از آجر. استفاده از لعاب سیلیس روی کف بنا، نیز از دیگر موضوع های جالب توجه در این کاخ است که به عنوان اندود کف به کار گرفته شده و از نظر رنگ و جنس و روش ساخت، موضوع قابل تأمل است.» بیان این نکات کافی است بدانیم که چه اثر فاخر و منحصر به فردی در این منطقه از ایران وجود دارد.

کشف سقف یکپارچه نانو تکنولوژی در کاخ کوروش آنچه در مورد اهمیت این اثر در حاشیه خلیج فارس نوشته شود، دال بر سیادت دریایی ایرانیان و سند مهم خلیج فارس است، مضافاً به این که شکوه و جلال و نقشه و پلان این قصر در این مکان، از ویژگی های خاصی برخوردار است که در سایر کاخ های هم زمان دیده نمی شود و این مهم را مخصوصاً در پو

In [49]:
len(text.split(' '))

9768905

In [50]:
text = text[:(len(text)//10)]

In [51]:
len(text)

5230345

In [52]:
len(text.split(' '))

1004072

In [53]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ £§ª«¬­¯°±²¶¹»¼½¾ÉÖ×ØÙÚÛßàáâäåæçèéëíîñóôõö÷øüāăčēěğīıńōœřŠšūɑɒəɛɪʁʒʾʿˈ˓ˢ́̃ΑΔΚΠΩάήίαβδεζηθικλμνξοπρςστυφχωόύАБВКМНПУЮавеийклмнопрстуцчьёћԑԱԲԳԴԵԹԿՀՄՅՈՍՎՏՓաբգդեզէթիլխծկհղմյնշոչպջռստրցւփքְִַָֹּׁבדהוחיכמןנסרש،؛؟ءآأؤإئابةتثجحخدذرزسشصضطظعغـفقكلمنهوىئًٌٍَُِّْ٠١٢٣٤٥٦٧٨٩٪٫٬ٰٲٴپځچڕژکګگۀیە۰۱۲۳۴۵۶۷۸۹ਖਦਨਲਵਸਾਿੋაეიოუṛṭἈἘῖ ‍‎‏–—‘’“”†•…′″‹›ℓ™↑→⇔∇∈−∕√∩∪≈≘≝≠≡≤≥⊆⌠⟨⟩《》いがしたとのもろァアィイウェエオカガキクグコサザシジスズゼソゾタッツトドナニネハバビピフブプベペボマミャュユラリルレロンヴ・ー人作兵別唐団型大奇女娘子屯巨幻德心戦撃教査火点無燐特狐獣班種系肉草行調豆豌超進遇遭邪鎧隊食駐鶴黄개기내년늑대리머소속손송예우의중지진𐭍𐭑
582


In [54]:
chinese_char = '種'
index = text.find(chinese_char)
print("chinese character found at index: ", index)


chinese character found at index:  1441039


In [55]:
print(text[index-200:index+200])

ن ها ندارند و به همین دلیل، نحوه تکثیر آن ها هنوز برای انسان ها نامشخص است. اجساد تایتان ها بلافاصله پس از مرگ به سرعت به بخار تبدیل می شود. دمای بدن تایتان ها بالاست. برخی از تایتان ها را غیرعادی (奇行種 Kikō-shū) می نامند چون برخلاف دیگر تایتان ها عمل می کنند.

تایتان غول پیکر(超大型巨人 Chō-ōgata Kyojin) (Colossal Titan)
بزرگترین تایتانی که تا به حال دیده شده است. (با طولی نزدیک به ۶۰ متر) این تایتان ا


In [56]:
text = regex.sub(r'[^\p{Arabic}\s0-9]+', ' ', text)


In [57]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 0123456789 ءآأؤإئابةتثجحخدذرزسشصضطظعغفقكلمنهوىي٠١٢٣٤٥٦٧٨٩٪٫٬ٲٴپځچڕژکګگۀیە۰۱۲۳۴۵۶۷۸۹ 
87


In [58]:
print(text[142000:145000])

انسته اند یک سقف با طول ۴۰ متر و عرض ۱۰ متر به صورت یکپارچه و صاف با ضخامت حدودا  ۵۶ سانتیمتر بسازند  می توانیم این سقف را به عنوان یکی از عجایب جهان محسوب بنماییم 
ساخت بتن سبک ۲۵۰۰ سال پیش در ایران

بتن سبک برای اولین بار در جهان توسط کشور ما ساخته و در ۲۵۰۰ سال پیش در زمان هخامنشیان در کاخ کوروش در استان بوشهر در شهر برازجان مورد استفاده قرار گرفت  برای ساخت این نوع بتن  محصول از پیش ساخته شده را که با صمغ ترکیب نموده بودند  با استفاده از موادی مانند خون حیوانات و شاخ بز و گوسفند که از قبل ساییده شده بود ترکیب نموده و با کف حاصل بتن سبک را تولید می کردند  همچنین با کشف این اثر ملی که برای اولین بار توسط آقای دکتر علی اکبر سرفراز پدر باستان شناسی ایران کشف گردید می توان به صورت یقین اعلام داشت مخترع سیمان نیز  کشور ایران بوده و برای اولین بار در همان کاخ کوروش از آن استفاده گردیده است 



عنوان مقاله  مایکل ای  جکسون

مایکل آنتونی جکسون  متولد ۱۹۶۳ یک دانشمند کامپیوتر و مشاور رایانش خصوصی در لندن  انگلستان می باشد  وی همچنین یک محقق نیمه وقت در آزمایشگاه تحقیقاتی ای تی اند تی فلورهام

In [59]:
n = int(0.9*len(text)) 
train_data = text[:n]
val_data = text[n:]

In [60]:
class textdataset(Dataset):
    def __init__(self, text, vocab, seq_len):
        
        self.vocab = vocab
        
        stoi = { ch:i for i,ch in enumerate(vocab) }
        itos = { i:ch for i,ch in enumerate(vocab) }
        self.encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
        self.decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

        self.text = torch.tensor(self.encode(text), dtype=torch.long)
        
        self.seq_len = seq_len
        

    def __len__(self):
        return len(self.text)

    
    def __getitem__(self, index):
        i = index
        if(index>=(len(self.text)-self.seq_len)):
            i = (len(self.text)-self.seq_len) - 1
        inpt = self.text[i:i+self.seq_len]
        outpt = self.text[i+1:i+1+self.seq_len]

        return inpt, outpt


In [61]:
seq_len = 64
train_dataset = textdataset(train_data, vocab=chars, seq_len=seq_len)
val_dataset = textdataset(val_data, vocab=chars, seq_len=seq_len)

In [62]:
train_dataset[0]

(tensor([38, 45, 47, 20, 45,  1, 44, 41, 20, 43, 46,  1,  1, 65, 20, 28, 33, 20,
         46, 74,  1, 20, 30, 44, 45, 74,  1, 70, 74, 43, 74, 70, 74, 46,  0,  0,
         65, 20, 28, 33, 20, 46, 74,  1, 20, 30, 44, 45, 74,  1, 70, 74, 43, 74,
         70, 74, 46,  1, 21, 46,  1,  1, 20, 30]),
 tensor([45, 47, 20, 45,  1, 44, 41, 20, 43, 46,  1,  1, 65, 20, 28, 33, 20, 46,
         74,  1, 20, 30, 44, 45, 74,  1, 70, 74, 43, 74, 70, 74, 46,  0,  0, 65,
         20, 28, 33, 20, 46, 74,  1, 20, 30, 44, 45, 74,  1, 70, 74, 43, 74, 70,
         74, 46,  1, 21, 46,  1,  1, 20, 30, 44]))

In [63]:
len(train_dataset)

4662816

In [64]:
print(train_dataset.encode("سلامم چطوری"))
print(train_dataset.decode(train_dataset.encode("سلامم چطوری")))

[32, 43, 20, 44, 44, 1, 67, 36, 47, 30, 74]
سلامم چطوری


In [65]:
# def dataloader(val, batch_num, seq_size):
#     if (val):
#         dataset=val_data
#     else:
#         dataset=train_data
    
#     last_possible_idx =  len(dataset) - seq_size
#     batch_start_idx = [random.randint(0,last_possible_idx) for _ in range(batch_num)]
    
#     inputs = []
#     outputs = []
#     for i in batch_start_idx:
#         inpt = dataset[i:i+seq_size]
#         outpt = dataset[i+1:i+1+seq_size]
#         inputs.append(inpt)
#         outputs.append(outpt)
    
#     return inputs, outputs

# x, y = dataloader(True, 4, 5)
# print(x)
# print(y)     

In [66]:
train_dataloader = DataLoader(train_dataset, batch_size=32,  shuffle=True, num_workers=2  )
val_dataloader = DataLoader(val_dataset, batch_size=32,  shuffle=True, num_workers=2  )

i = 0
# Iterate over the dataloader to access your data in batches
for batch in train_dataloader:
    i +=1
    if (i== 10):
        break
    print(batch)
print(i)

[tensor([[74,  1, 20,  ..., 20, 74, 45],
        [ 1,  1, 74,  ..., 28,  1,  0],
        [ 1,  1, 77,  ...,  0,  0, 38],
        ...,
        [ 1,  1,  0,  ..., 33,  1, 44],
        [ 1, 20, 45,  ...,  1, 44, 44],
        [ 1, 44, 41,  ..., 74, 70,  1]]), tensor([[ 1, 20, 44,  ..., 74, 45,  1],
        [ 1, 74, 70,  ...,  1,  0,  0],
        [ 1, 77, 83,  ...,  0, 38, 45],
        ...,
        [ 1,  0,  0,  ...,  1, 44, 74],
        [20, 45, 28,  ..., 44, 44, 70],
        [44, 41, 20,  ..., 70,  1, 70]])]
[tensor([[28,  1,  0,  ..., 74, 30, 74],
        [ 1,  0,  0,  ..., 47, 30, 24],
        [44, 74,  1,  ..., 44, 23, 47],
        ...,
        [ 1, 20, 74,  ..., 20, 43, 74],
        [ 1, 77, 85,  ..., 31,  1, 74],
        [41, 44,  1,  ..., 43, 44, 74]]), tensor([[ 1,  0,  0,  ..., 30, 74, 28],
        [ 0,  0,  0,  ..., 30, 24,  1],
        [74,  1, 21,  ..., 23, 47, 43],
        ...,
        [20, 74, 45,  ..., 43, 74, 23],
        [77, 85, 77,  ...,  1, 74, 70],
        [44,  1, 47,

In [67]:
vocab_size = len(chars)
seq_len = 64

In [68]:
device = "cuda"
if (not torch.cuda.is_available()):
    device = 'cpu'
print(device)

cuda


In [69]:
class Head(nn.Module):
    def __init__(self, head_size, embed_dim, dropout=0.0):
        super().__init__()
        self.key = nn.Linear(embed_dim, head_size, bias=False)
        self.query = nn.Linear(embed_dim, head_size, bias=False)
        self.value = nn.Linear(embed_dim, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(seq_len, seq_len)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores 
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size, embed_dim, dropout = 0.0):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, embed_dim, 0.2) for _ in range(num_heads)])
        self.proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out




class FeedFoward(nn.Module):
    def __init__(self, embed_dim, dropout = 0.0):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embed_dim, 4 * embed_dim),
            nn.ReLU(),
            nn.Linear(4 * embed_dim, embed_dim),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

    
class Block(nn.Module):
    # Transformer block

    def __init__(self, embed_dim, n_head):
        # embed_dim: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = embed_dim // n_head
        self.sa = MultiHeadAttention(n_head, head_size, embed_dim, 0.2)
        self.ffwd = FeedFoward(embed_dim)
        self.ln1 = nn.LayerNorm(embed_dim)
        self.ln2 = nn.LayerNorm(embed_dim)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [70]:
class LLM(nn.Module):
    def __init__(self, vocab_size, embed_dim, n_layer, n_head):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        self.position_embedding = nn.Embedding(seq_len, embed_dim)
        self.blocks = nn.Sequential(*[Block(embed_dim, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(embed_dim) # final layer norm
        self.lm_head = nn.Linear(embed_dim, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T=seq_len) tensor of integers
        # T=seq_len and C=embed_dim
        tok_emb = self.token_embedding(idx) # (B,T,C)
        pos_emb = self.position_embedding(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last seq_len tokens
            idx_cond = idx[:, -seq_len:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [75]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split, dataloader in zip(['train', 'val'], [train_dataloader, val_dataloader]):
        losses = []
        num_batches = 50
        for i, (X, Y) in enumerate(dataloader):
            X=X.to(device)
            Y=Y.to(device)
            logits, loss = model(X, Y)
            losses.append(loss.item())
            if i + 1 >= num_batches:
                break
        out[split] = np.array(losses).mean()

    model.train()
    return out

model = LLM(vocab_size, embed_dim = 32, n_layer=4, n_head=4)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
learning_rate=1e-3
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

iterations=10
for iter in range(10):
    if iter % 3 == 0 or iter == iterations - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    num_batches = 500
    for i, (x, y) in enumerate(train_dataloader):
        x=x.to(device)
        y=y.to(device)
        # evaluate the loss
        logits, loss = model(x, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i + 1 >= num_batches:
            break

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(train_dataset.decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.058199 M parameters
step 0: train loss 4.6631, val loss 4.6507
step 3: train loss 2.2081, val loss 2.5195
step 6: train loss 1.9530, val loss 2.3411
step 9: train loss 1.8680, val loss 2.2472

اثضالی حراط یاص رابا کند ندر بشه شد سیاد و که اثثته  زبا شخیتر شورگی او شسه راندین ۱۹ خورض  به دالایت او در دهمت  آو صرب امننیخی و وجه دی۳ ا حسهر حرامم موبلا ۲۷۵ میل بتر مرحدی  درنشک در به جور حزن محصد    نیاز  را زشدماران علام منا خوام  شوده اننست نگرامعی نوزه ارست  هانیهانگیر زمنداد  هست ایده  پلاقه تجار های درآن در از گرف خواله ژوازی عن سیل تو جمتات دادامنی به میروجه ای کردگرگی  شدیربان شباعال می شون اصه تند  شاط که بانشد 




عنوان مقال  اساس کلادی 
 این شدندهای ۱۹۶ یک که طول آن 2۴۲۱  



عنوان مقاله  جباندسته منند بانرتراری بارم به دکی ساد  تمی کلی بویان اولد  به به هسبریگ شه با        آی ب۲۴  بی۰۰۱ میلادی کند که پرسسش  ۱۹۳۵  متوع وی هونه  هر   آسرشگاه منبازاه شده کند خالات ترطای به اشدل 

۳۸۱ اچ هر   یموک چیامر


آرزفادیل گیردره اس  گرستگیه شاحده     نیاز خوستم در که اراه  ۱۸۰۰

نظرن سنای

In [77]:
iterations=20
for iter in range(iterations):
    if iter % 3 == 0 or iter == iterations - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    num_batches = 500
    for i, (x, y) in enumerate(train_dataloader):
        x=x.to(device)
        y=y.to(device)
        # evaluate the loss
        logits, loss = model(x, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i + 1 >= num_batches:
            break

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(train_dataset.decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

step 0: train loss 1.7144, val loss 2.1038
step 3: train loss 1.7193, val loss 2.0892
step 6: train loss 1.6838, val loss 2.0754
step 9: train loss 1.6624, val loss 2.0589
step 12: train loss 1.6766, val loss 2.0456
step 15: train loss 1.6644, val loss 2.0242
step 18: train loss 1.6225, val loss 2.0118
step 19: train loss 1.6381, val loss 2.0163

سیتفاحی نشان پای  یا رسنان ایالات متحده آمریکا بود 




عنوان مقاله  یواس دی پییوند  ویپنی سنای آذریک با سرقه ايی آتوز تی کماری  آن  بود که طول آن می باشد  


عنوان مقاله  جلس دراف ایراکان حدوتر سناتور ایالات متحده آمریکان هشته در برای عضو حاف دمور شود  اجس از جموره  این این طور بلق عصدم همسمی نب شهر معرکزی آها خاس نویل گر سیاسی ایجاد با جینست گیرهای درندین فرگاتانش وی از اثی اارسام شوکرندند  بایاض الش بین کل انتشای آی ویژه شاختن    مهنطق عارکثیر دروه پوه به غا زوده این گزن  ۲٫۹۸۲ تنه به و پیه

تبت این جایش اتاد پراهسلی متنین ۹۱۰۰ قادواون به ۱۰ قصاب تعالف است 
در بتاید وماند به مصرا یاللت تامیده شبکند 

پاهلیسل  پمپیشنگی اول ممی تواب تا متأث ذ

In [78]:
iterations=20
for iter in range(iterations):
    if iter % 3 == 0 or iter == iterations - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    num_batches = 1000
    for i, (x, y) in enumerate(train_dataloader):
        x=x.to(device)
        y=y.to(device)
        # evaluate the loss
        logits, loss = model(x, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i + 1 >= num_batches:
            break

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(train_dataset.decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

step 0: train loss 1.6291, val loss 2.0265
step 3: train loss 1.6056, val loss 1.9952
step 6: train loss 1.6199, val loss 1.9957
step 9: train loss 1.5941, val loss 1.9944
step 12: train loss 1.5806, val loss 1.9623
step 15: train loss 1.6055, val loss 1.9614
step 18: train loss 1.5757, val loss 1.9599
step 19: train loss 1.5846, val loss 1.9691


جعادل و خاستم منوز اسکس  در دی از صدی  این مانتیعان پیمکند و بلاعه می کنند  در نگار چه به تهرالیس حای یافت  با محیقه بنجه از شده بسیان بلا بهی 


کحوبوم ولی را ۱۹۶۹ میلادیت سناتور ایالت ناآن در تأمی اینی بن نتیفاع شمال  سیسوم ونسی ها فیلم به ا راه مرکزش نصاحی محروبی مشکل باطوح جورتری صاعت و انتر به قرا ترینیخ فیزور این وی براد را دکتار تغییلیت این در سام گیزاردگان دره سال ۲٫۴۷ صداد از پهوان  نماه گرزاری نیم جهان بین بسیار به این ایریخت اخلعاط  یکی طی  جیز ایلن کتر ۱۷بلات ویار به کشرت استفاده بود  رئینان استسط می ملی ژاننگ است  ارمن 23  صحیحا دیگرا کردند 

رونای لاژی  توانات ۱۰ بیکی کوئیا مرت آمده و درک  در دس  این عنواله باشد ولی در سال ۲۰۱  

In [79]:
iterations=50
for iter in range(iterations):
    if iter % 3 == 0 or iter == iterations - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    num_batches = 1000
    for i, (x, y) in enumerate(train_dataloader):
        x=x.to(device)
        y=y.to(device)
        # evaluate the loss
        logits, loss = model(x, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i + 1 >= num_batches:
            break

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(train_dataset.decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

step 0: train loss 1.5806, val loss 1.9445
step 3: train loss 1.6065, val loss 1.9379
step 6: train loss 1.5512, val loss 1.9347
step 9: train loss 1.5888, val loss 1.9559
step 12: train loss 1.5602, val loss 1.9285
step 15: train loss 1.5450, val loss 1.9255
step 18: train loss 1.5674, val loss 1.9127
step 21: train loss 1.5402, val loss 1.9198
step 24: train loss 1.5717, val loss 1.8985
step 27: train loss 1.5476, val loss 1.8953
step 30: train loss 1.5433, val loss 1.8985
step 33: train loss 1.5311, val loss 1.8833
step 36: train loss 1.5290, val loss 1.9063
step 39: train loss 1.5394, val loss 1.8918
step 42: train loss 1.5321, val loss 1.9160
step 45: train loss 1.5366, val loss 1.8979
step 48: train loss 1.5131, val loss 1.8851
step 49: train loss 1.5461, val loss 1.8817





عنوان مقاله  اچ ام اس کانفت  ۱۸۴ 

اچ ام اس ارسر  آب ۱۷۸  یک کشتی است که طول آن می باشد 



عنوان مقاله  رلیسک

مئیس اف الیجی با  پی استون   تبکی فضایه  سرل نکه نزد آمبیا اختلار تا بادولت یک  محسوبل از به سا

In [80]:
iterations=30
for iter in range(iterations):
    if iter % 3 == 0 or iter == iterations - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    num_batches = 1000
    for i, (x, y) in enumerate(train_dataloader):
        x=x.to(device)
        y=y.to(device)
        # evaluate the loss
        logits, loss = model(x, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i + 1 >= num_batches:
            break

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(train_dataset.decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

step 0: train loss 1.5329, val loss 1.8906
step 3: train loss 1.5365, val loss 1.8949
step 6: train loss 1.4869, val loss 1.8834
step 9: train loss 1.4844, val loss 1.8845
step 12: train loss 1.5093, val loss 1.8618
step 15: train loss 1.5079, val loss 1.8829
step 18: train loss 1.5338, val loss 1.8747
step 21: train loss 1.5116, val loss 1.8831
step 24: train loss 1.5013, val loss 1.8819
step 27: train loss 1.5419, val loss 1.8846
step 29: train loss 1.5029, val loss 1.8787


پی ک سناتور سابق عضو حزب دموکرات ایالت اممرده ای را یکملز ژوئی چشمنی های هنگزیدرایی محلات است  تراک در ۱۳۰  سال ۱۸۱۲ میلادی بود که طول آن بود  این کشتی در سال ۱۸۴۴ ساخته طول آن می بایلات میلاد دهد  سختت وجود که طول آن بود 



عنوان مقاله  فوات وی رسوم عمل اب در سپس امورد  ۵۹۹ هجهان سازی علیف او تاوئیه می داند 

فنای مروش تحقیقات محمدعه خداده خنط محیط نیابد به نوع مصرف برای پوس نشای عبدری خم کابکرنان علدان قمومت عنقلاع هموا به رای آی ستانیما به عنشی یک در است ارمرد و برای است 

دوم دانشگیت طبق دمان یشتری به طول آن

In [92]:
print(m.generate(context, max_new_tokens=2000)[0].tolist())
print(train_dataset.decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

[0, 0, 45, 47, 30, 1, 28, 30, 1, 37, 30, 40, 23, 1, 20, 47, 1, 28, 30, 1, 32, 23, 20, 30, 1, 15, 45, 1, 28, 40, 23, 1, 70, 46, 1, 33, 27, 34, 74, 23, 1, 1, 47, 1, 74, 70, 74, 1, 20, 31, 1, 33, 21, 70, 46, 1, 20, 31, 1, 38, 44, 47, 43, 1, 45, 37, 30, 1, 20, 41, 28, 20, 44, 1, 27, 47, 28, 1, 30, 74, 20, 45, 74, 1, 21, 46, 1, 41, 20, 44, 31, 21, 1, 20, 45, 28, 20, 31, 46, 1, 72, 30, 1, 44, 26, 43, 1, 46, 20, 1, 28, 30, 1, 20, 28, 31, 20, 30, 1, 44, 74, 1, 72, 74, 30, 28, 1, 0, 0, 47, 38, 28, 1, 46, 20, 74, 1, 65, 20, 74, 28, 45, 1, 46, 20, 74, 1, 20, 25, 30, 20, 1, 1, 20, 32, 23, 1, 20, 74, 45, 1, 21, 46, 1, 65, 40, 23, 46, 1, 21, 20, 1, 21, 30, 20, 74, 1, 1, 44, 26, 43, 46, 1, 44, 26, 20, 40, 37, 46, 1, 21, 20, 33, 28, 1, 1, 70, 20, 30, 46, 1, 20, 74, 45, 1, 20, 45, 23, 41, 20, 43, 1, 1, 15, 21, 74, 28, 1, 1, 1, 20, 31, 1, 38, 45, 47, 20, 45, 1, 32, 20, 45, 20, 23, 47, 30, 1, 28, 30, 1, 32, 20, 21, 41, 1, 38, 35, 47, 1, 26, 31, 21, 1, 28, 44, 47, 70, 30, 20, 23, 1, 20, 74, 20, 43, 20, 23

In [110]:
batch_size = 32

In [111]:
@torch.no_grad()
def perpl_rouge():
    model.eval()
    r1 = []
    r2 = []
    p = []
    num_batches = 50
    for i, (X, Y) in enumerate(val_dataloader):
        X=X.to(device)
        Y=Y.to(device)
        logits, loss = model(X, Y)
        p.append(2**(loss.item()))
        
    
        context = X[0].unsqueeze(0)
        pred = train_dataset.decode(m.generate(context, max_new_tokens=64)[0].tolist())
        print('expected: ', train_dataset.decode(context[0].tolist()))
        print('predicted: ' , pred)
        for j in range(batch_size):
            context = X[j].unsqueeze(0)
            pred = train_dataset.decode(m.generate(context, max_new_tokens=64)[0].tolist())
            scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
            scores = scorer.score(train_dataset.decode(context[0].tolist()), pred)
            r1.append(scores['rouge1'].recall)
            r2.append(scores['rouge2'].recall)
        
        if i + 1 >= num_batches:
            break
            
    r1 = np.array(r1).mean()
    r2 = np.array(r2).mean()
    p = np.array(p).mean()

    return r1,r2,p

print(perpl_rouge())

expected:  یو بیشتر منطقی و تابع نظم و قاعده  ویژه ای هستند که قبلا  توسط ه
predicted:  یو بیشتر منطقی و تابع نظم و قاعده  ویژه ای هستند که قبلا  توسط هر مسرمولگی گذارد کلیلین نمادار شرکت های قراری متمام کوربوگرای ها
expected:  ت به آن ها به عنوان شکلی از تنبیه می نگرند  این حکم به کشور تضمی
predicted:  ت به آن ها به عنوان شکلی از تنبیه می نگرند  این حکم به کشور تضمیکر به عبه شناسی عبارت جایز در مورد ماندار این ساختر نوشته بوده ا
expected:  ن حلال  به قطبیت انعطاف پذیر آب وابسته است که آن هم مستقیما  به 
predicted:  ن حلال  به قطبیت انعطاف پذیر آب وابسته است که آن هم مستقیما  به عنوان قرار ازدا بویال جابرجی می شد  به ویندرد

در محبق دوره از م
expected:  زی الگوریتم تصادفی بسیار ساده تری را بر اساس تعمیم الگوریتم تصاد
predicted:  زی الگوریتم تصادفی بسیار ساده تری را بر اساس تعمیم الگوریتم تصادت بخش کمکن می کند  در طر اکی لالمی های نیز جری در سلطان با یا نج
expected:  ل اگر نمونه ۳ بخشی قابل حل باشد  مسئله افراز  1  نیز در گراف   ب
predicted:  ل اگر نمونه ۳ بخشی قابل حل باشد  مسئله افراز